In [43]:
import pandas as pd
df=pd.read_csv("synthetic_heterogeneous_pack_scaled/matters_A.csv")


In [44]:
df.head()

,matter_id,client_ref,title,practice_area,opened_on,lead_attorney,estimated_value
0,MAT-1001,CL-1001,Healthcare Solutions Inc v. ACME Corp - Employ...,Tax,20/05/2025,Daniel Park,7902
1,MAT-1002,CL-1002,Aerospace Solutions Inc - Regulatory Inquiry,Environmental,02/03/22,Michael Brown,6815
2,MAT-1003,CL-1003,Finance Group LLC - Master Services Agreement ...,Environmental,29/08/2025,Hannah Lee,41K
3,MAT-1004,CL-1004,Construction Systems - Regulatory Inquiry,Litigation,1722501280,Hannah Lee,"$14,790.00"
4,MAT-1005,CL-1005,Pharmaceuticals Holdings Corp - Contract Breach,Environmental,09/09/2021,Ayesha Khan,43K
